In [1]:
from braket.tracking import Tracker

### Executa circuito síncrono

In [ ]:
def run_circuit_sync(profile, backend, circuit):
    """Executa um circuito de forma síncrona e retorna os resultados."""
    try:
        tracker = Tracker()
        config = load_config(profile)
        shots = config["shots"]

        device = AwsDevice(backend.value)

        # Só estimamos custo para QPUs
        if device.type == "QPU":
            estimated_cost = estimate_cost(backend, shots)
            if estimated_cost is not None:
                print(f"⚠️ Custo estimado: ${estimated_cost:.6f}")
                confirm = input("Deseja continuar? (s/n): ").strip().lower()
                if confirm != 's':
                    print("❌ Execução cancelada pelo usuário.")
                    return None

        tracker.start()
        print(f"Executando (SÍNCRONO) com {shots} shots no perfil '{profile.value}' no backend '{backend.name}'")

        task = device.run(circuit, shots=shots)
        result = task.result()  # Aguarda a execução

        tracker.stop()
        print("📊 Estatísticas de Uso:", tracker.quantum_tasks_statistics())
        print(f"💰 Custo real: ${tracker.qpu_tasks_cost() + tracker.simulator_tasks_cost():.6f}")

        return result.measurement_counts

    except Exception as e:
        print(f"Erro: {e}")

### Executa circuito assíncrono

In [ ]:
def run_circuit_async(profile, backend, circuit):
    """Executa um circuito de forma assíncrona e retorna o ARN da task."""
    try:
        tracker = Tracker()
        config = load_config(profile)
        shots = config["shots"]

        device = AwsDevice(backend.value)

        # Só estimamos custo para QPUs
        if device.type == "QPU":
            estimated_cost = estimate_cost(backend, shots)
            if estimated_cost is not None:
                print(f"⚠️ Custo estimado: ${estimated_cost:.6f}")
                confirm = input("Deseja continuar? (s/n): ").strip().lower()
                if confirm != 's':
                    print("❌ Execução cancelada pelo usuário.")
                    return None

        tracker.start()
        print(f"Executando (ASSÍNCRONO) com {shots} shots no perfil '{profile.value}' no backend '{backend.name}'")

        task = device.run(circuit, shots=shots)
        tracker.stop()

        print("📌 Task ARN:", task.id)
        return task.id  # Retorna o ARN da task para verificação posterior

    except Exception as e:
        print(f"Erro: {e}")

### Checa status task 

In [2]:
def check_task_status(task_arn):
    """Verifica o status da task e retorna o resultado se estiver pronto."""
    try:
        task = AwsDevice.get_task(task_arn)
        status = task.state()

        print(f"🔄 Status da Task {task_arn}: {status}")

        if status == "COMPLETED":
            result = task.result()
            print("✅ Execução completa! Resultados:", result.measurement_counts)
            return result.measurement_counts
        elif status in ["FAILED", "CANCELLED"]:
            print(f"❌ Task {task_arn} falhou ou foi cancelada.")
        else:
            print(f"⌛ Task {task_arn} ainda em execução...")
        
        return None  # Ainda não está pronto

    except Exception as e:
        print(f"Erro ao verificar status: {e}")